##### Imports

In [27]:
import sys, os
from digraph import *
from equivalenceClass import *
from topoSorts import *
from algorithmTime import *
from recursiveFormula import *
from naiveFormula import *
#TODO: Mantener una consistencia entre el Camel Cases o el Snake Case, revisar que se usa en Python

### Testing functions

In [28]:
def assertEquivalenceClassesForNode(dag: nx.DiGraph, feature_node, all_topo_sorts: List[List[Any]], timing_dict: Dict[str, Dict[str, float]]):
    
    nodes_classification = {}
    unr_roots = classifyNodes(dag, feature_node, nodes_classification)
    hasher = TopoSortHasher(nodes_classification)

    # Naive approach
    start_time = time.time()
    naiveClassesSizes = naiveEquivalenceClassesSizes(all_topo_sorts, feature_node, hasher)
    end_time = time.time()
    timing_dict[feature_node]['Naive Formula'] = end_time - start_time

    # Recursive approach
    start_time = time.time()
    recursiveClassesSizes = recursiveEquivalenceClassesSizes(dag, unr_roots, hasher, feature_node, nodes_classification)
    end_time = time.time()
    timing_dict[feature_node]['Recursive Formula'] = end_time - start_time

    timing_dict[feature_node]['Number of equivalence classes'] = len(naiveClassesSizes.keys())
    
    # Assert that each equivalence class has the same number of elements.

    naiveEqClasses = len(naiveClassesSizes.keys())
    recursiveEqClasses = len(recursiveClassesSizes.keys())
    if naiveEqClasses != recursiveEqClasses and numberOfEquivalenceClasses(dag, feature_node) != naiveEqClasses:
        raise AssertionError(f"The number of equivalence classes is different. \n Naive Approach: {naiveEqClasses}, Recursive Approach: {recursiveEqClasses} \n Feature Node: {feature_node}")

    assertTopoSortsAndEquivalenceClasses(dag, feature_node, recursiveClassesSizes)

    for eqClassHash in naiveClassesSizes.keys():
        clSize1 = naiveClassesSizes[eqClassHash][1]
        clTopo1 = naiveClassesSizes[eqClassHash][0]
        try: 
            clSize2 = recursiveClassesSizes[eqClassHash][1]
            clTopo2 = recursiveClassesSizes[eqClassHash][0]
        except KeyError:
            raise AssertionError(f"The equivalence class {eqClassHash} is not present in the recursive approach. \n Naive Approach: Topo {clTopo1}, Size {clSize1} \n Feature Node: {feature_node}")
        if (clSize1 != clSize2):
            raise AssertionError(f"The sizes of the equivalence classes are not equal. \n Naive Approach: Topo {clTopo1}, Size {clSize1} \n Recursive Approach: Topo {clTopo2}, Size {clSize2} \n Feature Node: {feature_node}")

#TODO: Find a better algorithm than all_topological_sorts, it takes too much time. In the paper they mention a dynamic programming approach, maybe implement that. This takes too much time.

def assertEquivClassesForDag(dag: nx.DiGraph, nodesToEvaluate = None, allSorts = None) -> Dict[str, float]:
    timing_dict = {}
    
    # Measure time for all topological sorts
    start_time = time.time()
    all_topo_sorts = allSorts if allSorts != None else list(nx.all_topological_sorts(dag))
    assert len(all_topo_sorts) == allTopoSorts(dag)
    end_time = time.time()
    timing_dict['Time Of Topological Sorts'] = end_time - start_time
    timing_dict['Number of Topological Sorts'] = len(all_topo_sorts)
    
    nodesToEvaluate = nodesToEvaluate if nodesToEvaluate != None else list(dag.nodes)
    for node in nodesToEvaluate:
            timing_dict[node] = {}
            assertEquivalenceClassesForNode(dag, node, all_topo_sorts, timing_dict)
    
    return timing_dict

## Examples

In [29]:
numNodes = 7

emptyTestGraph = emptyGraph(numNodes)
resEmptyGraph = assertEquivClassesForDag(emptyTestGraph)

naiveBayesTest = naiveBayes(numNodes)
resNaiveBayes = assertEquivClassesForDag(naiveBayesTest)

lengthOfPath = 4
naiveBayesWithPathTest = naiveBayesWithPath(numNodes, lengthOfPath)
resNaiveBayesWithPath = assertEquivClassesForDag(naiveBayesWithPathTest)

numberOfPaths = 3
numNodes = 3
multiplePathsTest = multiplePaths(numNodes, numberOfPaths)
resMultiplePaths = assertEquivClassesForDag(multiplePathsTest)

numLevels = 2
branchingFactor = 3
treeTest = balancedTree(numLevels, branchingFactor)
resTree = assertEquivClassesForDag(treeTest)

def test_allTopos(graph):
    all_topos = allTopoSorts(graph)
    all_topo_sorts = list(nx.all_topological_sorts(graph))
    assert all_topos == len(all_topo_sorts), "allTopos and all_topological_sorts have different lengths"

for graph in [emptyTestGraph, naiveBayesTest, naiveBayesWithPathTest, multiplePathsTest, treeTest]:
    test_allTopos(graph)


## Experimentation

### Auxiliary Functions

In [30]:
printEnabled = True
def disablePrint():
    global printEnabled
    if printEnabled:
        sys._jupyter_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
        printEnabled = False

def enablePrint():
    global printEnabled
    printEnabled = True
    sys.stdout.close()
    sys.stdout = sys._jupyter_stdout


### Multiple paths with different lengths

In [31]:
def timeMultiplePathsGraphs(numPaths, pathLength, startFrom = 1):
    graphsResults = {}
    for i in range(startFrom,numPaths+1):
        for j in range(startFrom,pathLength+1):
            graphToEvaluate = multiplePaths(i, j)
            #drawGraph(graphToEvaluate)
            nodesToEvaluate = list(range(0, j))
            #print(f'{i} Paths, {j} Length' + str(nodesToEvaluate))
            graphsResults[f'{i} Paths, {j} Length'] = measure_graph_data(graphToEvaluate, nodesToEvaluate)

    return graphsResults
    

numberOfPaths = 5
pathLenght = 6
graph = multiplePaths(numberOfPaths, pathLenght)
        
allTopos = allTopoSorts(graph)
res = timeRecursiveFunctionFor(graph, list(range(0, pathLenght)))

### Time depending on Equivalence Classes

In [32]:
def timeMultipleBalancedTrees(numLevels, branchingFactor = 2, startLevels = 1, starBranching = 2):
    graphsResults = {}
    for i in range(startLevels,numLevels+1):
        for j in range(starBranching,branchingFactor+1):
            graphToEvaluate = balancedTree(i, j)
            #drawGraph(graphToEvaluate)
            leafNode = [node for node in graphToEvaluate.nodes if isLeaf(node, graphToEvaluate)][0]
            pathToLeaf = orderedNodes(graphToEvaluate, nx.ancestors(graphToEvaluate, leafNode)) + [leafNode]
            print(f'{i} Levels, {j} Branching' + str(pathToLeaf))
            graphsResults[f'{i} Levels, {j} Branching'] = measure_graph_data(graphToEvaluate, pathToLeaf)
            #print()

    return graphsResults

numLevels = 4
branchingFactor = 2

res = timeMultipleBalancedTrees(numLevels, branchingFactor, numLevels, branchingFactor)
